# RAY

In [1]:
import ray
import pandas as pd

ray.init()

2022-05-27 13:24:00,418	INFO services.py:1462 -- View the Ray dashboard at http://127.0.0.1:8265


RayContext(dashboard_url='127.0.0.1:8265', python_version='3.6.9', ray_version='1.12.0', ray_commit='f18fc31c7562990955556899090f8e8656b48d2d', address_info={'node_ip_address': '10.0.2.15', 'raylet_ip_address': '10.0.2.15', 'redis_address': None, 'object_store_address': '/tmp/ray/session_2022-05-27_13-23-56_475995_14352/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2022-05-27_13-23-56_475995_14352/sockets/raylet', 'webui_url': '127.0.0.1:8265', 'session_dir': '/tmp/ray/session_2022-05-27_13-23-56_475995_14352', 'metrics_export_port': 47871, 'gcs_address': '10.0.2.15:44541', 'address': '10.0.2.15:44541', 'node_id': 'f22bd92195efe388369e947f04ec77ec70e65123fd70445e8316218b'})

### Управление объектами в Object Store

In [2]:
y = "Hello world"
y_ref = ray.put(y)

print(y_ref)
print(ray.get(y_ref))

ObjectRef(00ffffffffffffffffffffffffffffffffffffff0100000001000000)
Hello world


### Tasks

In [3]:
@ray.remote(num_cpus=1)
def func(a: int) -> int:
    return a + 1

# аргумент автоматически добавляется в хранилище
res_ref = func.remote(2)
print(ray.get(res_ref))

# можно добавить вручную
y = 1
y_ref = ray.put(y)
res_ref = func.remote(y_ref)
print(ray.get(res_ref))

3
2


### Actors

In [4]:
# если нужен объект, который существует в единственном экземпляре
@ray.remote
class Actor:
    def __init__(self):
        self.d = dict()    

    def get(self, k: int) -> str:
        return self.d[k]

    def set(self, k: int, v: str):        
        self.d[k] = v


actor_ref = Actor.remote()
actor_ref.set.remote(2, "Hello")

res_ref = actor_ref.get.remote(2)
print(ray.get(res_ref))

Hello


### Datasets

In [5]:
ds = ray.data.range(10000)
ds = ray.data.read_csv("currencies.csv")
ds.show(5)

ds = ds.repartition(2)

ds_new = ds.map(
    lambda record: {"USD" : 2 * record["USD"] }
)
ds_new.show(5)

def _mapper(df: pd.DataFrame) -> pd.DataFrame:
    df["USD"] *= 2
    return df

ds_new = ds.map_batches(
    _mapper, 
    batch_format="pandas"
)
ds_new.show(5)

Map Progress:   0%|          | 0/2 [00:00<?, ?it/s]

{'DATE': datetime.date(2017, 4, 1), 'EUR': 59.8107, 'USD': 55.9606, 'GBP': 69.7605}
{'DATE': datetime.date(2017, 4, 4), 'EUR': 59.8953, 'USD': 56.1396, 'GBP': 70.3429}
{'DATE': datetime.date(2017, 4, 5), 'EUR': 60.2427, 'USD': 56.5553, 'GBP': 70.3548}
{'DATE': datetime.date(2017, 4, 6), 'EUR': 59.6948, 'USD': 55.894, 'GBP': 69.4986}
{'DATE': datetime.date(2017, 4, 7), 'EUR': 60.0827, 'USD': 56.4369, 'GBP': 70.3655}
{'USD': 111.9212}
{'USD': 112.2792}
{'USD': 113.1106}
{'USD': 111.788}
{'USD': 112.8738}


Map Progress: 100%|██████████| 2/2 [00:00<00:00, 21.88it/s]


{'DATE': datetime.date(2017, 4, 1), 'EUR': 59.8107, 'USD': 111.9212, 'GBP': 69.7605}
{'DATE': datetime.date(2017, 4, 4), 'EUR': 59.8953, 'USD': 112.2792, 'GBP': 70.3429}
{'DATE': datetime.date(2017, 4, 5), 'EUR': 60.2427, 'USD': 113.1106, 'GBP': 70.3548}
{'DATE': datetime.date(2017, 4, 6), 'EUR': 59.6948, 'USD': 111.788, 'GBP': 69.4986}
{'DATE': datetime.date(2017, 4, 7), 'EUR': 60.0827, 'USD': 112.8738, 'GBP': 70.3655}
